In [98]:
import os
import sklearn
import pandas as pd
import csv

In [99]:
n_gram_value = 5

In [235]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [264]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
   # possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [265]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [266]:
def writeToCsv(processed_list):
    with open("output-sreejita.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        #for row in processed_list:
        wr.writerows(processed_list)

In [267]:
'''def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            if(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):       
                    
                    # prune common words
                    if(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])

                    #prune unigrams 
                    elif(i==1 and word_number> word_count+1):                        
                        
                        #add the unigram to the list if the next word begins with lower case
                        if(possible_words[word_count+1][0].islower()):
                            preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])  
            word_count = word_count+1
    return preprocessed_list   ''' 

'def preprocessing(possible_words,doc_id):    \n    suffix_list = getListFromFiles("company-suffixes.txt")\n    prefix_list = getListFromFiles("company-prefixes.txt")    \n    common_words = getListFromFiles("common-words.txt")\n    preprocessed_list =[]    \n    marked_list=[]\n    word_number=len(possible_words)\n    for i in range(5,0,-1):\n        word_count=0\n        ngram_list = get_ngrams(possible_words,i)\n        for ngram in ngram_list:            \n            #print ngram   \n            \n            #if markup is both at the beginning or at the end of the word group accept and label as 1\n            if(len(ngram)>1 and \'<markup>\' in ngram[0] and \'</markup>\' in ngram[len(ngram)-1] and\n              all((word[0].isupper() or "markup" in word) for word in ngram) and \n              ["<markup>" in word for word in ngram].count(True)==1):\n                preprocessed_list.append([\' \'.join(ngram),doc_id,word_count,word_count+i-1,1])\n            \n            #get all

In [304]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1): 
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple)                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            elif(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple) 
                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):  
                    
                    #prune unigrams 
                    if(i==1):                        
                        if(ngram[0] not in suffix_list):
                            if(word_number> word_count+1 and ngram[0] not in common_words):
                                #add the unigram to the list if the next word begins with lower case
                                if(possible_words[word_count+1][0].islower()):
                                    preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])
                    
                    #prune common words
                    elif(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])                   

                    
                     
            word_count = word_count+1
    return preprocessed_list    

In [316]:
def generateFeatures(filename, processed_list,possible_words):
    
    company_suffixes = getListFromFiles("company-suffixes.txt")
    company_prefixes = getListFromFiles("company-prefixes.txt")    
    company_list=[]
    final_processed_list = []
    splittedFilename = filename.split("-")
    i=0
    print "possible_words- "  
    print possible_words
    for company in processed_list:
        i+=1
        #Feature 1 : name has any prefix or suffix of a company 
        #if company name has any one of the suffixes or prefixes set it to true 
        hasCompanyid=0        
        if(company[0].split()[-1] in company_suffixes or company[0].split()[0] in company_prefixes):
            hasCompanyid = 1
            company_list.append(company[0])            
        #company.append(hasCompanyid)
        
        
        #has Company name at the beginning ?
        #Feature 2: name occurs at the beginning of a company name
        hasCompanyNameFirst = 0
        l1= [x for x in company[0].split() if x not in company_prefixes]
        l2=[]
        for comp in company_list:    
            if(comp.split()[0] in company_prefixes):
                l2.append(" ".join(comp.split()[1:]))
            else:
                l2.append(comp)
        if(any(word in l2 for word in l1)):
            hasCompanyNameFirst = 1
        elif(any(company[0].split()[0]== word.split()[0] for word in company_list)):
            hasCompanyNameFirst = 1 
        
            
        
        #has Company name as a substring ?
        #Feature 3: name is a substring in the company name
        hasNameSubstring=0
        for comp in company_list:
            l1 = company[0].split()
            l2 = comp.split()
            if(len(set(l1)&set(l2)) > 0 ):
                hasNameSubstring=1       
       
         
        
        #Feature 4: name in filename
        isInFilename = 0
        splittedInstance = company[0].split()
        for sp in splittedInstance:
            if sp.lower() in splittedFilename:
                isInFilename = 1 
        
        
        #Feature 5-y - Check neibhouring words - check 3 words before and 3 words after
        starting_loc = company[2]
        ending_loc = company[3]
        words_before_starting_loc = []
        words_after_ending_loc = []
        j= starting_loc-1
        count = 1
        while j >= 0 and count <= 3:
            words_before_starting_loc.append(possible_words[company[2]-count])
            j-=1
            count+=1
        
        j = ending_loc+1
        count = 1
        while j < len(possible_words) and count <= 3:
            words_after_ending_loc.append(possible_words[company[3]+count])
            j+=1
            count+=1
        
        #Feature 5 - Check for word "based" before the company
        has_based = 0
        for word in words_before_starting_loc:
            if "based" in word.lower():
                has_based = 1
        
        #Feature 6 "'s" is after the company
        has_apostrophe_s = 0
        for word in words_after_ending_loc:
            if "'s" in word.lower():
                has_apostrophe_s = 1
        '''
        #Feature 7 "," is after/before the company
        has_comma = 0
        for word in words_after_ending_loc:
            if "," in word.lower():
                has_comma = 1
        
        for word in words_before_starting_loc:
            if "," in word.lower():
                has_comma = 1
        '''
        #Feature 8 designations before/after company
        desig = open('designations.txt').read().split('\n')
        desig_lower = [x.lower() for x in desig]
        has_desig = 0
        for word in words_after_ending_loc:
            if word.lower() in desig_lower:
                has_desig = 1
              
        for word in words_before_starting_loc:
            if word.lower() in desig_lower:
                has_desig = 1 
         
        #Feature 9 "said/told" in the before company
        has_said = 0
        for word in words_before_starting_loc:
            if "said" in word.lower() or "told" in word.lower():
                has_said = 1
        
        '''
        #Feature 10 Prune away Examples if previous/next word starts with a Capital
        surrounded_capital = 0
        if len(words_before_starting_loc) > 0:
            prev_word = words_before_starting_loc[0]
            if len(prev_word) > 0 and prev_word[0].isupper():
                surrounded_capital = 1
        if len( words_after_ending_loc) > 0:
            next_word = words_after_ending_loc[0]
            if len(next_word) > 0 and next_word[0].isupper():
                surrounded_capital = 1
        '''
          
                
        
        print "company:- " + company[0]
        print "words_after_ending_loc"
        print words_after_ending_loc
        print "words_before_starting_loc"
        print words_before_starting_loc
        
        company.extend([hasCompanyid, hasCompanyNameFirst, hasNameSubstring, isInFilename, has_based, has_apostrophe_s, has_desig, has_said])
        
            

In [336]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Training_1")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
#doc_id = 101
for file in file_list:
    relevant_filename = file[4:-4]
    doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    ''''processed_list.append(preprocessing(possible_words,doc_id))
    writeToCsv(processed_list)'''
    processed_list_for_doc = preprocessing(possible_words,doc_id)
    #print processed_list_for_doc
    generateFeatures(relevant_filename, processed_list_for_doc,possible_words)
    processed_list.extend(processed_list_for_doc)
    #writeToCsv(processed_list) 
  

possible_words- 
['<markup>Maanshan', 'Iron', 'And', 'Steel', 'Co</markup>', '600808', "China's", 'second-biggest', 'Hong', 'Kong-listed', 'steelmaker', 'said', '2006', 'profit', 'fell', '18', 'percent', 'because', 'of', 'lower', 'average', 'prices', 'and', 'higher', 'production', 'costs', 'Net', 'income', 'fell', 'to', '239', 'billion', 'yuan', '$309', 'million', 'or', '035', 'yuan', 'a', 'share', 'from', '291', 'billion', 'yuan', 'or', '045', 'yuan', 'a', 'share', 'a', 'year', 'ago', 'the', '<markup>Maanshan</markup>', 'Anhui', 'province-based', 'company', 'said', 'in', 'a', 'statement', 'to', 'the', 'Shanghai', 'Stock', 'Exchange', 'today', 'citing', 'international', 'accounting', 'standards', 'Sales', 'rose', '6', 'percent', 'to', '34', 'billion', 'yuan', 'It', 'will', 'pay', 'a', 'cash', 'dividend', 'of', '013', 'yuan', 'per', 'share', 'for', '2006', 'compared', 'with', '016', 'yuan', 'a', 'share', 'a', 'year', 'earlier', 'The', 'steelmaker', 'had', 'a', 'gross', 'profit', 'margin

company:- Just
words_after_ending_loc
['hours', 'before', 'the']
words_before_starting_loc
['weeks', 'few', 'a']
company:- Bally
words_after_ending_loc
['disclosed', 'in', 'a']
words_before_starting_loc
['filing', '11', 'Chapter']
company:- Commission
words_after_ending_loc
['that', 'it', 'had']
words_before_starting_loc
['Exchange', 'and', 'Securities']
possible_words- 
['<markup>Bank', 'of', 'Ireland', 'Plc</markup>', 'the', "nation's", 'biggest', 'mortgage', 'lender', 'said', 'earnings', 'growth', 'will', 'slow', 'as', 'rising', 'interest', 'rates', 'hurt', 'demand', 'for', 'home', 'loans', 'Earnings', 'per', 'share', 'will', 'rise', 'by', 'a', 'percentage', 'in', 'the', 'low', 'double', 'digits', 'in', 'the', 'year', 'to', 'the', 'end', 'of', 'March', '2008', 'Chief', 'Executive', 'Officer', 'Brian', 'Goggin', 'said', 'today', 'in', 'an', 'interview', 'Excluding', 'exceptional', 'gains', 'earnings', 'rose', '22', 'percent', 'in', 'the', 'year', 'ended', 'March', '31', '2007', 'Iris

possible_words- 
['<markup>Carrefour', 'SA<markup>', 'CA', "Europe's", 'biggest', 'retailer', 'must', 'find', 'ways', 'to', 'sell', 'real', 'estate', 'without', 'disturbing', 'its', 'operations', 'Chief', 'Executive', 'Officer', 'Jose', 'Luis', 'Duran', 'said', 'We', 'have', 'to', 'find', 'solutions', 'the', 'CEO', 'said', 'today', 'in', 'an', 'interview', 'during', 'a', 'conference', 'in', 'Barcelona', 'Spain', 'Duran', 'softened', 'his', 'stance', 'on', 'real', 'estate', 'after', 'strategy', 'talks', 'with', 'representatives', 'of', '<markup>Colony', 'Capital', 'LLC</markup>', 'and', '<markup>Groupe', 'Arnault</markup>', 'who', 'teamed', 'up', 'this', 'month', 'to', 'buy', 'a', '98', 'percent', 'stake', 'in', 'Paris-based', '<markup>Carrefour</markup>', 'The', 'CEO', 'has', 'resisted', 'property', 'sales', 'until', 'now', 'arguing', 'that', 'loss', 'of', 'control', 'over', 'rental', 'costs', 'would', 'erode', 'margins', 'We', 'all', 'agree', 'on', 'targets', '--', 'how', 'to', 'go', 

words_before_starting_loc
['metal', 'the', 'for']
company:- Zhang
words_after_ending_loc
['said', 'Most', 'of']
words_before_starting_loc
['2005', 'from', 'percent']
company:- Most
words_after_ending_loc
['of', 'the', 'new']
words_before_starting_loc
['said', 'Zhang', '2005']
company:- Chinese
words_after_ending_loc
['producers', 'including', '<markup>Jinchuan</markup>']
words_before_starting_loc
['nickel', "nation's", 'the']
company:- Jinchuan
words_after_ending_loc
['will', 'boost', 'overseas']
words_before_starting_loc
['including', 'producers', 'Chinese']
company:- Zhang
words_after_ending_loc
['in', 'an', 'interview']
words_before_starting_loc
['said', 'demand', 'rising']
possible_words- 
['China', 'may', 'need', 'to', 'import', '46', 'percent', 'of', 'its', 'crude', 'oil', 'needs', 'by', '2010', 'as', 'domestic', 'demand', 'surges', 'according', 'to', 'an', 'official', 'from', '<markup>China', 'Petroleum', 'And', 'Chemical', 'Corp</markup>', '600028', 'Rising', 'consumption', 'wi

possible_words- 
["Mexico's", 'Supreme', 'Court', 'took', 'preliminary', 'votes', 'to', 'curb', 'rights', 'broadcasters', 'had', 'gained', 'under', 'a', '2006', 'law', 'saying', 'it', 'gave', 'the', 'two', 'largest', 'companies', '<markup>Grupo', 'Televisa', 'SA</markup>', 'TLEVICPO', 'and', '<markup>TV', 'Azteca', 'SA</markup>', 'unfair', 'advantages', 'Automatically', 'letting', 'broadcasters', 'add', 'data', 'and', 'telecommunications', 'services', 'on', 'request', 'violates', 'the', "constitution's", 'fairness', 'provisions', 'judges', 'said', 'today', 'during', 'deliberations', 'in', 'Mexico', 'City', 'The', 'court', 'also', 'decided', 'that', 'auctioning', 'licenses', 'to', 'the', 'highest', 'bidder', 'is', 'unjustified', 'and', 'that', '20-year', 'license', 'terms', 'are', 'too', 'long', 'The', 'votes', 'are', 'the', 'latest', 'steps', 'by', 'the', 'court', 'to', 'reshape', 'radio', 'and', 'television', 'rules', 'set', 'by', 'the', '2006', 'law', 'If', 'confirmed', 'in', 'a', 'f

...

possible_words- 
['<markup>Scottish', 'And', 'Newcastle', 'Plc</markup>', 'which', 'is', 'in', 'a', 'legal', 'battle', 'with', '<markup>Carlsberg', 'A/S</markup>', 'CARLA', 'over', 'their', 'Russian', 'joint', 'venture', 'said', 'arbitration', 'of', 'the', 'dispute', 'will', 'be', 'resolved', 'within', 'six', 'months', 'The', 'final', 'award', 'shall', 'be', 'made', 'by', 'July', '3', '2008', 'Edinburgh-', 'based', '<markup>Scottish', 'And', 'Newcastle</markup>', 'said', 'today', 'in', 'a', 'statement', 'citing', 'a', 'letter', 'from', 'the', '<markup>Stockholm', 'Chamber', 'of', 'Commerce</markup>', 'to', 'both', 'brewers', '<markup>Carlsberg</markup>', 'does', 'not', 'expect', 'a', 'resolution', 'in', 'the', 'case', 'for', 'at', 'least', 'a', 'year', 'spokesman', 'Jens', 'Peter', 'Skaarup', 'said', '<markup>Scottish', 'And', 'Newcastle</markup>', 'the', 'largest', 'UK', 'beermaker', 'began', 'arbitration', 'in', 'October', 'after', '<markup>Carlsberg</markup>', 'and', '<markup>Heinek

['Exchange', 'composite', 'trading']
words_before_starting_loc
['New', 'in', 'pm']
company:- Stock Exchange
words_after_ending_loc
['composite', 'trading', 'The']
words_before_starting_loc
['York', 'New', 'in']
company:- Icelandic
words_after_ending_loc
['investment', 'company', '<markup>Baugur']
words_before_starting_loc
['by', 'takeover', 'possible']
company:- York
words_after_ending_loc
['store', 'Net', 'income']
words_before_starting_loc
['New', 'main', 'its']
company:- Net
words_after_ending_loc
['income', 'rose', 'to']
words_before_starting_loc
['store', 'York', 'New']
company:- Saks
words_after_ending_loc
['said', 'today', 'in']
words_before_starting_loc
['cents', '5', 'or']
company:- August
words_after_ending_loc
['opening', 'of', 'a']
words_before_starting_loc
['The', 'statement', 'a']
company:- Avenue
words_after_ending_loc
['store', 'spurred', 'sales']
words_before_starting_loc
['Fifth', 'the', 'at']
company:- US
words_after_ending_loc
["dollar's", 'decline', 'the']
words_be

possible_words- 
['<markup>Sun', 'Pharmaceutical', 'Industries', 'Ltd</markup>', 'SUNP', 'and', '<markup>Watson', 'Pharmaceuticals', 'Inc</markup>', 'WPI', 'agreed', 'to', 'hold', 'off', 'on', 'selling', 'a', 'generic', 'version', 'of', '<markup>Novartis', 'AG</markup>', 'NOVN', "'s", 'dementia', 'drug', 'Exelon', 'in', 'the', 'US', 'as', 'part', 'of', 'a', 'patent', 'litigation', 'settlement', 'The', 'two', 'companies', 'agreed', 'not', 'to', 'sell', 'the', 'drug', 'until', 'sometime', 'prior', 'to', 'the', 'expiration', 'of', 'the', 'patents', 'owned', 'by', '<markup>Novartis</markup>', 'Mumbai-based', '<markup>Sun', 'Pharmaceutical</markup>', 'and', 'Corona', 'California-based', '<markup>Watson</markup>', 'said', 'in', 'separate', 'statements', '<markup>Novartis</markup>', 'will', 'license', 'the', 'patents', 'to', 'the', 'companies', 'they', 'said', '<markup>Novartis</markup>', 'owns', 'two', 'patents', 'on', 'rivastigmine', 'tartrate', 'that', 'expire', 'in', '2012', 'and', '2014'

['largest', 'seven', 'the']
possible_words- 
['<markup>Virgin', 'America', 'Inc</markup>', 'the', 'low-cost', 'airline', 'partly', 'owned', 'by', 'UK', 'billionaire', 'Richard', 'Branson', 'wants', 'the', 'US', 'to', 'let', 'its', 'chief', 'executive', 'officer', 'stay', 'three', 'months', 'longer', 'so', 'the', 'company', 'has', 'time', 'to', 'replace', 'him', 'Finding', 'a', 'successor', 'for', 'CEO', 'Fred', 'Reid', 'has', 'proven', 'challenging', 'and', 'he', 'should', 'be', 'able', 'to', 'keep', 'working', 'until', 'Feb', '18', 'instead', 'of', 'Nov', '18', 'as', 'the', 'Transportation', 'Department', 'ordered', '<markup>Virgin', 'America</markup>', 'said', 'yesterday', 'in', 'a', 'petition', 'filed', 'in', 'Washington', '<markup>Virgin', 'America</markup>', 'agreed', 'to', 'replace', 'Reid', '57', 'after', 'the', 'department', 'said', 'last', 'December', 'it', 'was', 'worried', 'he', 'would', 'be', 'beholden', 'to', 'Branson', 'US', 'law', 'bars', 'non-citizens', 'from', 'exercis

['this', 'point', 'the']
words_before_starting_loc
['23', 'Nov', 'on']
company:- Christmas
words_after_ending_loc
['trading', 'period', 'Chief']
words_before_starting_loc
['the', 'to', 'up']
company:- Goyder
words_after_ending_loc
['said', 'in', 'the']
words_before_starting_loc
['Richard', 'Officer', 'Executive']
possible_words- 
['<markup>Wet', 'Seal', 'Inc</markup>', 'WTSLA', 'the', 'apparel', 'chain', 'for', 'teenage', 'girls', 'fell', 'the', 'most', 'in', 'more', 'than', 'three', 'years', 'in', 'US', 'trading', 'after', 'lowering', 'its', 'third-quarter', 'earnings', 'forecast', 'and', 'reporting', 'an', 'unexpected', 'drop', 'in', 'September', 'sales', 'The', 'stock', 'dropped', '74', 'cents', 'or', '19', 'percent', 'to', '$325', 'at', '4', 'pm', 'for', 'the', 'biggest', 'decline', 'since', 'August', '2004', 'It', 'has', 'slumped', '51', 'percent', 'this', 'year', '<markup>Foothill', 'Ranch</markup>', 'California-based', 'retailer', 'said', 'yesterday', 'that', 'third-quarter', 'n

In [337]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [338]:
["markup" in word for word in ngram].count(True)==2

True

In [339]:
import pycountry
country_names = []
for country in pycountry.countries:
    country_names.append(country.name)
    if hasattr(country,'common_name'):
        country_names.append(country.common_name)
    if hasattr(country, 'official_name'):
        country_names.append(country.official_name)

country_names.append("America")
country_names.append("Korea")
country_names.append("South Korea")
country_names.append("North Korea")
country_names.append("U.S.")
country_names.append("Russia")
country_names.append("Siberia")

if "Russia" in country_names:
    print "YES"
    
continent_list = ['Asia', 'Africa', 'South America', 'North America', 'Antarctica', 'Europe', 'Australia']

YES


In [340]:
cities = []
with open('world-cities.csv', 'r') as inp:
        for row in csv.reader(inp):
            if(row and row[3] != "" and row[0]!="name"):
                cities.append(row[0])

cities.append('Bombay')
cities.append('New York')
if "Compton" in cities:
    print "YES"

us_states = open('us-states.txt').read().split('\n')
ethni_list = open('ethnicities.txt').read().split('\n')

YES


In [341]:
desig = open('designations.txt').read().split('\n')
desig_lower = [x.lower() for x in desig]

print desig_lower

['chief executive officer', 'chief financial officer', 'co-chief executive', 'chief investment officer', 'analyst', 'ceo', 'cio', 'cto', 'cfo', 'manager', 'director', 'president', 'vice-president', 'vice president', 'chief', 'executive', 'officer', 'officers', 'analysts', 'managers', 'directors', 'co-chief', 'co chief', 'commissioner', 'minister', 'magistrate', 'judge', 'chairman']


In [342]:
pruned_pos = open('pruned_pos.txt').read().split('\n')
stock_x = open('stock-exchange-list.txt').read().split('\n')
unigram_prune = open('unigram-prune-neg.txt').read().split('\n')
special_char = [';', ',', ':', '?']
prune = open('pruning-neg.txt').read().split('\n') 
prune_lower = [x.lower() for x in prune]
suffixes = open('company-suffixes.txt').read().split('\n')
prefixes = open('company-prefixes.txt').read().split('\n')
rows = 0
print len(processed_list)
for doc in processed_list:
    rows += len(doc)


7030


In [343]:
print pruned_pos
new_proc_list = []
pruned_list = []

for inst in processed_list:
    if inst[0] not in pruned_pos: 
        if inst[0] in country_names or inst[0] in continent_list or inst[0] in cities or inst[0] in stock_x or inst[0] in us_states:
            pruned_list.append(inst)
        elif "'s" in inst[0]:
            pruned_list.append(inst)
        elif inst[0].endswith('n') and inst[0][:-1] in country_names or str in continent_list:
            pruned_list.append(inst)
        elif "based" in inst[0]:
            pruned_list.append(inst)
        elif any(d in inst[0].lower().split() for d in desig_lower):
            pruned_list.append(inst)
        elif any(p in inst[0].lower().split() for p in prune_lower):
            pruned_list.append(inst)
        elif '(' in inst[0] or ')' in inst[0]:
            pruned_list.append(inst)
        elif any(c in inst[0] for c in special_char):
            pruned_list.append(inst)
        elif inst[0].startswith("A ") or inst[0].startswith("An ") or inst[0].startswith("And "):
             pruned_list.append(inst)
        elif len(inst[0].split()) == 1:
            if len(inst[0]) == 1:
                pruned_list.append(inst)
            elif inst[0] in unigram_prune or inst[0] in ethni_list:
                pruned_list.append(inst)
            elif inst[0] == inst[0].upper():
                pruned_list.append(inst)
            elif inst[0] in suffixes or inst[0] in prefixes:
                pruned_list.append(inst)
            else:
                new_proc_list.append(inst)
        else:
            new_proc_list.append(inst)
    else:
        new_proc_list.append(inst)
            
                
print len(pruned_list)
for eg in pruned_list:
    if eg[4] == 1:
        print "HOLY MOLY!"
#writeToCsv(processed_list)
print len(new_proc_list)
print len(pruned_list) + len(new_proc_list)
#print pruned_list



['ACE', "Sam's Club", 'ICBC', 'KKR', 'CSN', 'Tata', 'Wheeling', "River's Edge Pharmaceuticals LLC", "River's Edge", 'RWE', 'Fremont', 'News Corp', 'GM', 'Liberty', 'Financial Services Authority', 'FSA', 'NZOG', 'Norilsk', 'NTP', 'MPC', 'PPR', 'THQ', 'Nokia', 'RCS', 'Intercontinental Exchange Inc', 'EON', 'RHB', 'WCM', 'Macclesfield', 'Carrefour', 'KPI', 'TXU', 'ECI', 'BP', 'TNK-BP', 'Imperial', 'Zug', 'TDF', 'TMK', "Ivernia Inc's", 'SAIC', 'GE', 'PICC', 'Saku', 'Winterthur', 'ZKB', "Rupert Murdoch's News Corp", 'PIK', 'VTB', 'UBS', 'IGT', 'BHP', 'BBH', 'China Banking Regulatory Commission', 'Compton', 'Centennial', 'NPD', 'HSBC', "Victoria's Secret", 'MAN', 'MBIA', 'Freeport', 'MGIC', 'PMI', 'Stillwater', "Dr Reddy's Laboratories Ltd", "Royal Mail's", 'Cia Siderurgica Nacional SA', 'Cia Siderurgica Nacional', 'Vector Hospitality Plc', 'Vector', 'Victory']
3170
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY M

In [344]:
'''thefile = open('pruned_pos.txt', 'w')
for item in pruned_pos:
    #print item
    thefile.write("%s\n" % item)
len(pruned_pos)
print pruned_list'''

'thefile = open(\'pruned_pos.txt\', \'w\')\nfor item in pruned_pos:\n    #print item\n    thefile.write("%s\n" % item)\nlen(pruned_pos)\nprint pruned_list'

In [345]:
with open("train_pruned.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in new_proc_list:
        writer.writerow(val)